In [1]:
# %run packages.ipynb

In [1]:
from pdf_files import get_fullpath_of_pdf_files

In [2]:
from pdf_documents import get_pdf_documents

In [3]:
from document_util import get_split_documents

In [4]:
from embedding_function import get_embedding_function

In [5]:
from chroma_db import create_db, get_similar_docs

In [6]:
from chain import get_chain_output

In [7]:
from match import run_text_match

In [8]:
# path = "./data/docs/literature1" # 1 good 
# path = "./data/docs/literature6" # 6 good 
# path = "./data/docs/literature9" # 9 good
path = "./data/docs/literature"  # all

chunk_size=1500
chunk_overlap=100
persist_directory = 'data/db/vector/huggingface_index_all'
collection_name = 'paper_index'

In [9]:
embedding_function = get_embedding_function()

### The following 4 lines only need to be run once (after the pdf is loaded for the first time)

In [20]:
pdf_files = get_fullpath_of_pdf_files(path)

In [21]:
pdf_docs = get_pdf_documents(pdf_files)

./data/docs/literature/shih2012.pdf is not identified! Using other strategy!!
./data/docs/literature/EPBC1.pdf is not identified! Using other strategy!!
./data/docs/literature/su2015.pdf is not identified! Using other strategy!!
./data/docs/literature/tujula2007.pdf is not identified! Using other strategy!!
LLM strategy failed!!./data/docs/literature/tujula2007.pdf Please manually add it!!
./data/docs/literature/xu2011.pdf is not identified! Using other strategy!!
./data/docs/literature/kahn2015.pdf is not identified! Using other strategy!!


In [ ]:
split_docs = get_split_documents(pdf_docs, chunk_size, chunk_overlap)

In [ ]:
create_db(split_docs,embedding_function,collection_name,persist_directory)

## Query

In [10]:
query = "However, the China's cadre evaluation system is different from the voting system in the Western countries"

In [11]:
similar_docs = get_similar_docs(query, 5, embedding_function, collection_name, persist_directory)

Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index2


In [12]:
output = get_chain_output(query,similar_docs)

In [13]:
final_list = run_text_match(output,query,similar_docs, embedding_function)

No result was found, Using embedding searching strategy!!!
The 0 sentence
Sentence: Empirically, the literature linking China’s cadre eval- uation system and growth produces highly ambiguous and problematic results
Source:{'author': 'VICTOR SHIH and CHRISTOPHER ADOLPH and MINGXING LIU', 'year': '2012', 'title': 'Getting Ahead in the Communist Party: Explaining the Advancement of Central Committee Members in China', 'journal': 'American Political Science Review'}
Score:0.6213207978549135
========.
========.
The 1 sentence
Sentence: After the founding of the People’s Republic of China, the CCP introduced formal methods of evaluating cadres, which were formally linked to their promotion
Source:{'author': 'VICTOR SHIH and CHRISTOPHER ADOLPH and MINGXING LIU', 'year': '2012', 'title': 'Getting Ahead in the Communist Party: Explaining the Advancement of Central Committee Members in China', 'journal': 'American Political Science Review'}
Score:0.6769256411963147
========.
========.
The 2 sent